# Radhika Mardikar, Xinxin Mo

## Step 1:
Looking at the KEGG pathway, we will select 4 enzymes from the glycolysis, TCA cycle and pentose phosphate cycle. 
Glycolysis: hexokinase 1, phosphoglucose isomerase, phosphofructokinase, fructose-bisphosphate aldolase (https://www.ebi.ac.uk/interpro/potm/2004_2/Page2.htm)
TCA: citrate synthase, aconitase, isocitrase dehydrogenase, alpha-ketoglurate (https://www.news-medical.net/life-sciences/Krebs-Cycle-Enzymes.aspx)
pentose phosphate: transketolase, transaldolase, lactonase, phosphopentose isomerase (https://mcb.berkeley.edu/labs/krantz/mcb102/lect_S2008/MCB102-SPRING2008-LECTURE5-PENTOSE.pdf)

In [17]:
# import packages
from Bio import Entrez
from Bio import SeqIO
import prettytable as pt
import sqlite3

In [18]:
Entrez.email = "rmardikar@berkeley.edu"
# first row is glycolysis enzyme, second is pentose phosphate cycle, third is TCA
enzymelist = ["2.7.1.1", "5.4.2.2", "3.1.3.11", "5.3.1.9",
              "5.3.1.9", "3.1.3.11", "1.1.1.49", "1.1.1.44",
              "1.1.1.37", "1.2.4.1", "1.1.1.27", "1.8.1.4"]

organismlist = ['Homo sapiens', 'Drosophila melanogaster', "Escherichia coli"]
idlist, idlist2 = [], []

for org in organismlist:
    for enzyme in enzymelist:
        handle = Entrez.esearch(db="gene",
                                term = org + '[ORGN] AND ' + enzyme, 
                                idtype = 'acc', 
                                sort='relevance',
                                retmax=1)
        #print(org + '[ORGN]' + enzyme)
        record = Entrez.read(handle)
        idlist.append(record["IdList"])
        
        handle2 = Entrez.esearch(db="nucleotide",
                                term = org + '[ORGN] AND ' + enzyme, 
                                idtype = 'acc', 
                                sort='relevance',
                                retmax=1)
        record2 = Entrez.read(handle2)
        idlist2.append(record2["IdList"])


## Step 2

Create database as well as tables, including genes table, pathway table, enzyme table, pathway-enzyme table, gene-enzyme table.

In [21]:
# create a database
# connect to database
conn = sqlite3.connect('my.db')
c = conn.cursor()

# create pathway table
c.execute('CREATE TABLE pathway (name TEXT, description TEXT);')
# create enzymes table
c.execute('CREATE TABLE enzymes (name TEXT, enzyme_number TEXT, pathway_name TEXT);')
# create genes table
c.execute('CREATE TABLE genes (iD TEXT, name TEXT, ecnum TEXT, organism TEXT, chromosome TEXT, start INT, end INT, sequence TEXT);')
# create associate table
c.execute('CREATE TABLE pathway_enzymes (name TEXT, enzyme_number TEXT, pathway_name TEXT, description TEXT);')
# create associate table
c.execute('CREATE TABLE gene_enzymes (enzyme_name TEXT, enzyme_number TEXT, pathway_name TEXT, id TEXT, organism TEXT, chromosome TEXT, start INT, end INT);')

# insert values into pathway table
c.executemany('INSERT INTO pathway(name, description) VALUES(?, ?)',
         [('Glycolysis', 'A cycle that makes ATP'),
          ('Pentose Phosphate', 'Generates NADPH'),
          ('TCA', 'Inside mitochondria and results in ATP')])  
conn.commit()
#c.execute('SELECT * FROM pathway')

# Step 3

Fetch the data and store the useful data into tables. Here we fetch "gene" and "nucleotide". Because the sequences are too long, we display a sample result below.

In [22]:
# fetch data for genes table and enzymes table
rows = []
namelist = []

for i in range(len(idlist)):
    handle = Entrez.efetch(db="gene", id = idlist[i], rettype = 'gb', retmode = 'text', retnum=1)
    #tempHandle = handle.readlines()
    #print(handle.read())
    if i < 25:
        for line in handle.readlines():
            if 'Name' in line:
                name = line[line.find('Name')+len('Name:')+1:line.find('[')]
                organism = line[line.find('[')+1:line.find(']')]
                #print(name)
                #print(organism)
            if 'Annotation' in line:
                anno = line[line.find('Annotation')+len('Annotation:')+1:-1]
                annoList = line.split()
                chromosome = annoList[2]
                #print(annoList)
                startEndSplit = annoList[4].split('..')
                #print(startEndSplit)
                start = (startEndSplit[0])[1:-1]
                end = (startEndSplit[1])[0:-1]
                #print(int(start), int(end))
            if 'ID' in line:
                iD = int(line.split(' ')[-1])
                #print(iD)
    else:
        for line in handle.readlines():
            if 'Escherichia coli' in line:
                name = line[:line.find('[')]
                organism = 'Escherichia coli'
                #print(name)
                #print(organism)
            if 'Annotation' in line:
                anno = line[line.find('Annotation')+len('Annotation:')+1:-1]
                annoList = line.split()
                chromosome = '-'
                #print(annoList)
                startEndSplit = annoList[2].split('..')
                #print(startEndSplit)
                start = (startEndSplit[0])[1:-1]
                end = (startEndSplit[1])[0:-1]
                #print(int(start), int(end))
            if 'ID' in line:
                iD = int(line.split(' ')[-1])
                #print(iD)
    
    handle2 = Entrez.efetch(db="nucleotide", id = idlist2[i], rettype = 'fasta', retmode = 'text', retnum=1)
    #print(handle2.read())
    ecnum = enzymelist[i%12]
    description = '-'
    sequence = handle2.read()
    rows.append((iD, name, ecnum, organism, chromosome, start, end, sequence))  

**sample information fetched from database** You can find the whole data at the end of this notebook.

1. HK1
Official Symbol: HK1 and Name: hexokinase 1 [Homo sapiens (human)]
Other Aliases: HK-ta, HK1-tb, HK1-tc, HKD, HKI, HMSNR, HXK1, RP79, hexokinase, HK1
Other Designations: hexokinase-1; brain form hexokinase; glycolytic enzyme; hexokinase IR; hexokinase type I
Chromosome: 10; Location: 10q22.1
Annotation: Chromosome 10 NC_000010.11 (69269991..69401882)
MIM: 142600
ID: 3098

>AK294428.1 Homo sapiens cDNA FLJ56506 complete cds, highly similar to Hexokinase-1 (EC 2.7.1.1)
GAGGAGCCGCCGAGCAGCCGCCGGAGGACCACGGCTCGCCAGGGCTGCGGAGGACCGACCGTCCCCACGC
CTGCCGCCCCGCGACCCCGACCGCCAGCATGATCGCCGCGCAGCTCCTGGCCTATTACTTCACGGAGCTG
AAGGATGACCAGGTCAAAAAGATTGACAAGTATCTCTATGCCATGCGGCTCTCCGATGAAACTCTCATAG
ATATCATGACTCGCTTCAGGAAGGAGATGAAGAATGGCCTCTCCCGGGATTTTAATCCAACAGCCACAGT
CAAGATGTTGCCAACATTCGTAAGGTCCATTCCTGATGGCTCTGAAAAGGGAGATTTCATTGCCCTGGAT
CTTGGTGGGTCTTCCTTTCGAATTCTGCGGGTGCAAGTGAATCATGAGAAAAACCAGAATGTTCACATGG
AGTCCGAGGTTTATGACACCCCAGAGAACATCGTGCACGGCAGTGGAAGCCAGCTTTTTGATCATGTTGC
TGAGTGCCTGGGAGATTTCATGGAGAAAAGGAAGATCAAGGACAAGAAGTTACCTGTGGGATTCACGTTT
TCTTTTCCTTGCCAACAATCCAAAATAGATGAGGCCATCCTGATCACCTGGACAAAGCGATTTAAAGCGA
GCGGAGTGGAAGGAGCAGATGTGGTCAAACTGCTTAACAAAGCCATCAAAAAGCGAGGGGACTATGATGC
CAACATCGTAGCTGTGGTGAATGACACAGTGGGCACCATGATGACCTGTGGCTATGACGACCAGCACTGT
GAAGTCGGCCTGATCATCGGCACTGGCACCAATGCTTGCTACATGGAGGAACTGAGGCACATTGATCTGG
TGGAAGGAGACGAGGGGAGGATGTGTATCAATACAGAATGGGGAGCCTTTGGAGACGATGGATCATTAGA
AGACATCCGGACAGAGTTTGACAGGGAGATAGACCGGGGATCCCTCAACCCTGGAAAACAGCTGTTTGAG
AAGATGGTCAGTGGCATGTACTTGGGAGAGCTGGTTCGACTGATCCTAGTCAAGATGGCCAAGGAGGGCC
TCTTATTTGAAGGGCGGATCACCCCGGAGCTGCTCACCCGAGGGAAGTTTAACACCAGTGATGTGTCAGC
CATCGAAAAGAATAAGGAAGGCCTCCACAATGCCAAAGAAATCCTGACCCGCCTGGGAGTGGAGCCGTCC
GATGATGACTGTGTCTCAGTCCAGCACGTTTGCACCATTGTCTCATTTCGCTCAGCCAACTTGGTGGCTG
CCACACTGGGCGCCATCTTGAACCGCCTGCGTGATAACAAGGGCACACCCAGGCTGCGGACCACGGTTGG
TGTCGACGGATCTCTTTACAAGACGCACCCACAGTATTCCCGGCGTTTCCACAAGACTCTAAGGCGCTTG
GTGCCAGACTCCGATGTGCGCTTCCTCCTCTCGGAGAGTGGCAGCGGCAAGGGGGCTGCCATGGTGACGG
CGGTGGCCTACCGCTTGGCCGAGCAGCACCGGCAGATAGAGGAGACCCTGGCTCATTTCCACCTCACCAA
GGACATGCTGCTGGAGGTGAAGAAGAGGATGCGGGCCGAGATGGAGCTGGGGCTGAGGAAGCAGACGCAC
AACAATGCCGTGGTTAAGATGCTGCCCTCCTTCGTCCGGAGAACTCCCGACGGGACCGAGAATGGTGACT
TCTTGGCCCTGGATCTTGGAGGAACCAATTTCCGTGTGCTGCTGGTGAAAATCCGTAGTGGGAAAAAGAG
AACGGTGGAAATGCACAACAAGATCTACGCCATTCCTATTGAAATCATGCAGGGCACTGGGGAAGAGCTG
TTTGATCACATTGTCTCCTGCATCTCTGACTTCTTGGACTACATGGGGATCAAAGGCCCCAGGATGCCTC
TGGGCTTCACGTTCTCATTTCCCTGCCAGCAGACGAGTCTGGACGCGGGAATCTTGATCACGTGGACAAA
GGGTTTTAAGGCAACAGACTGCGTGGGCCACGATGTAGTCACCTTACTAAGGGATGCGATAAAAAGGAGA
GAGGAATTTGACCTGGACGTGGTGGCTGTGGTCAACGACACAGTGGGCACCATGATGACCTGTGCTTATG
AGGAGCCCACCTGTGAGGTTGGACTCATTGTTGGGACCGGCAGCAATGCCTGCTACATGGAGGAGATGAA
GAACGTGGAGATGGTGGAGGGGGACCAGGGGCAGATGTGCATCAACATGGAGTGGGGGGCCTTTGGGGAC
AACGGGTGTCTGGATGATATCAGGACACACTACGACAGACTGGTGGACGAATATTCCCTAAATGCTGGGA
AACAAAGGTATGAGAAGATGATCAGTGGTATGTACCTGGGTGAAATCGTCCGCAACATCTTAATCGACTT
CACCAAGAAGGGATTCCTCTTCCGAGGGCAGATCTCTGAGACGCTGAAGACCCGGGGCATCTTTGAGACC
AAGTTTCTCTCTCAGATCGAGAGTGACCGATTAGCACTGCTCCAGGTCCGGGCTATCCTCCAACAGCTAG
GTCTGAATAGCACCTGCGATGACAGTATCCTCGTCAAGACAGTGTGCGGGGTGGTGTCCAGGAGGGCCGC
ACAGCTGTGTGGCGCAGGCATGGCTGCGGTTGTGGATAAGATCCGCGAGAACAGAGGACTGGACCGTCTG
AATGTGACTGTGGGAGTGGACGGGACACTCTACAAGCTTCATCCACACTTCTCCAGAATCATGCACCAGA
CGGTGAAGGAACTGTCACCAAAATGTAACGTGTCCTTCCTCCTGTCTGAGGATGGCAGCGGCAAGGGGGC
CGCCCTCATCACGGCCGTGGGCGTGCGGTTACGCACAGAGGCATAGCATTAGCTGCTTCCTCCCCTCCTG
GCACCCACTGTGGCCTGGCATCGCATCGTGGTGTGTCAATGCCACAAAATCGTGTGTCCGTGGAACCAGT
CCTAGCCGCGTGTGACAGTCTTGCATTCTGTTTGTCTCGT

Insert values into enzymes and pathway tables.

In [23]:
# insert values into enzymes table
ezrows = []
for i in rows:
    namelist.append(i[1])
for x in range(0, 12):
    if x <= 3: 
        y = 'Glycolysis'
    elif (x > 3) and (x <= 7):
        y = 'Pentose Phosphate'
    else:
        y = 'TCA'
    temp = (namelist[x], enzymelist[x], y)
    ezrows.append(temp)
    #print(temp)
#print(len(ezrows))
c.executemany('INSERT INTO enzymes values (?,?,?)', ezrows)
conn.commit()

In [24]:
# insert values into genes table
#print(len(rows))
c.executemany('INSERT INTO genes values (?,?,?,?,?,?,?,?)', rows)
conn.commit()

In [25]:
# import genes table
c.execute("SELECT * FROM genes;")
# print table
tb = pt.PrettyTable()
tb.field_names = ["gene id", "EC number", "gene name", "organism", "chromosome", "start", "end"]

counter1 = 0
for tu in c.fetchall():
    tb.add_row(tu[:-1])
print(tb)

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------------------------+------------+----------+-----------+
| gene id  |                                                                               EC number                                                                               | gene name |               organism              | chromosome |  start   |    end    |
+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------------------------+------------+----------+-----------+
|   3098   |                                                                             hexokinase 1                                                                              |  2.7.1.1  |       

In [26]:
# import enzymes table
c.execute("SELECT * FROM enzymes;")
tb = pt.PrettyTable()
tb.field_names = ["enzyme name", "enzyme number", "pathway"]
# print enzymes table
counter = 0
for tu in c.fetchall():
    tb.add_row(tu)
    counter += 1
print(tb)

+--------------------------------------------+---------------+-------------------+
|                enzyme name                 | enzyme number |      pathway      |
+--------------------------------------------+---------------+-------------------+
|               hexokinase 1                 |    2.7.1.1    |     Glycolysis    |
|           phosphoglucomutase 1             |    5.4.2.2    |     Glycolysis    |
|         fructose-bisphosphatase 1          |    3.1.3.11   |     Glycolysis    |
|       glucose-6-phosphate isomerase        |    5.3.1.9    |     Glycolysis    |
|       glucose-6-phosphate isomerase        |    5.3.1.9    | Pentose Phosphate |
|         fructose-bisphosphatase 1          |    3.1.3.11   | Pentose Phosphate |
|     glucose-6-phosphate dehydrogenase      |    1.1.1.49   | Pentose Phosphate |
|      phosphogluconate dehydrogenase        |    1.1.1.44   | Pentose Phosphate |
|      phosphoglycerate dehydrogenase        |    1.1.1.37   |        TCA        |
| py

Here we print out the tables from the "my.db" database.

In [27]:
# import pathway table
c.execute("SELECT * FROM pathway;")
tb = pt.PrettyTable()
tb.field_names = ["pathway name", "description"]
# print pathway table
for tu in c.fetchall():
    tb.add_row(tu)
print(tb)

+-------------------+----------------------------------------+
|    pathway name   |              description               |
+-------------------+----------------------------------------+
|     Glycolysis    |         A cycle that makes ATP         |
| Pentose Phosphate |            Generates NADPH             |
|        TCA        | Inside mitochondria and results in ATP |
+-------------------+----------------------------------------+


## Step 3 

Insert values into the associate tables.

In [28]:
# join pathway table and enzymes table

sql = "SELECT enzymes.name, enzyme_number, pathway_name, description\
        FROM enzymes\
        JOIN pathway ON pathway.name = enzymes.pathway_name"
c.execute(sql)
pathway_enzymesList = c.fetchall()
c.executemany('INSERT INTO pathway_enzymes values (?,?,?,?)', pathway_enzymesList)

# print the associate pathway_enzymes table
tb = pt.PrettyTable()
tb.field_names = ["enzyme name", "EC number", "pathway name", "description"]
# print pathway table
for tu in pathway_enzymesList:
    tb.add_row(tu)
print(tb)

+--------------------------------------------+-----------+-------------------+----------------------------------------+
|                enzyme name                 | EC number |    pathway name   |              description               |
+--------------------------------------------+-----------+-------------------+----------------------------------------+
|               hexokinase 1                 |  2.7.1.1  |     Glycolysis    |         A cycle that makes ATP         |
|           phosphoglucomutase 1             |  5.4.2.2  |     Glycolysis    |         A cycle that makes ATP         |
|         fructose-bisphosphatase 1          |  3.1.3.11 |     Glycolysis    |         A cycle that makes ATP         |
|       glucose-6-phosphate isomerase        |  5.3.1.9  |     Glycolysis    |         A cycle that makes ATP         |
|       glucose-6-phosphate isomerase        |  5.3.1.9  | Pentose Phosphate |            Generates NADPH             |
|         fructose-bisphosphatase 1     

In [33]:
# join genes table and enzymes table
sql = "SELECT enzymes.name,  enzyme_number, pathway_name, iD, organism, chromosome, start, end\
        FROM enzymes\
        JOIN genes ON genes.ecnum = enzymes.enzyme_number"
c.execute(sql)
gene_enzymesList = c.fetchall()
#print(gene_enzymesList)
c.executemany('INSERT INTO gene_enzymes values (?,?,?,?,?,?,?,?)', gene_enzymesList)

# print the table
tb = pt.PrettyTable()
tb.field_names = ["enzyme name", "EC number", "pathway name", "ID", "organism", "chromosome", "start", "end"]
# print pathway table
for tu in gene_enzymesList:
    tb.add_row(tu)
print(tb)


+--------------------------------------------+-----------+-------------------+----------+-------------------------------------+------------+----------+-----------+
|                enzyme name                 | EC number |    pathway name   |    ID    |               organism              | chromosome |  start   |    end    |
+--------------------------------------------+-----------+-------------------+----------+-------------------------------------+------------+----------+-----------+
|               hexokinase 1                 |  2.7.1.1  |     Glycolysis    |   3098   |         Homo sapiens (human)        |     10     | 6926999  |  69401882 |
|               hexokinase 1                 |  2.7.1.1  |     Glycolysis    |  39988   | Drosophila melanogaster (fruit fly) |     3L     | 1784691  |  17849601 |
|               hexokinase 1                 |  2.7.1.1  |     Glycolysis    |  45875   | Drosophila melanogaster (fruit fly) |     X      |  958567  |  9589813  |
|           phos

## Appendix

Here is the data we pulled from the web browser.

In [34]:
rows = []
namelist = []

for i in range(len(idlist)):
    handle = Entrez.efetch(db="gene", id = idlist[i], rettype = 'gb', retmode = 'text', retnum=1)
    #tempHandle = handle.readlines()
    print(handle.read())


1. HK1
Official Symbol: HK1 and Name: hexokinase 1 [Homo sapiens (human)]
Other Aliases: HK-ta, HK1-tb, HK1-tc, HKD, HKI, HMSNR, HXK1, RP79, hexokinase, HK1
Other Designations: hexokinase-1; brain form hexokinase; glycolytic enzyme; hexokinase IR; hexokinase type I
Chromosome: 10; Location: 10q22.1
Annotation: Chromosome 10 NC_000010.11 (69269991..69401882)
MIM: 142600
ID: 3098



1. PGM1
Official Symbol: PGM1 and Name: phosphoglucomutase 1 [Homo sapiens (human)]
Other Aliases: CDG1T, GSD14
Other Designations: phosphoglucomutase-1; PGM 1; glucose phosphomutase 1
Chromosome: 1; Location: 1p31.3
Annotation: Chromosome 1 NC_000001.11 (63593276..63660245)
MIM: 171900
ID: 5236



1. FBP1
Official Symbol: FBP1 and Name: fructose-bisphosphatase 1 [Homo sapiens (human)]
Other Aliases: FBP
Other Designations: fructose-1,6-bisphosphatase 1; D-fructose-1,6-bisphosphate 1-phosphohydrolase 1; FBPase 1; growth-inhibiting protein 17; liver FBPase
Chromosome: 9; Location: 9q22.32
Annotation: Chromoso


1. Pgd
Official Symbol: Pgd and Name: Phosphogluconate dehydrogenase [Drosophila melanogaster (fruit fly)]
Other Aliases: Dmel_CG3724, 6-PGD, 6-Pgd, 6-pgd, 6PGD, 6PGDH, 6Pgd, 6Pgdh, 6pgdh, A7, CG3724, Dmel\CG3724, EG:87B1.4, N1, PGD, Pdg, l(1)2Dc, l(1)2De, l(1)A7, l(1)G0385, l(1)N1, l(1)N3[90], l(1)Pgd, l(1)Pgd-A, l35, pgd
Other Designations: phosphogluconate dehydrogenase; 6-phosphogluconate; 6-phosphogluconate dehydrogenase; CG3724-PA; CG3724-PB; NADP-6 phosphogluconate dehydrogenase; Pgd-PA; Pgd-PB; dehydrogenase
Chromosome: X; Location: 1-0.6 cM
Annotation: Chromosome X NC_004354.4 (2145136..2148566)
ID: 31185



1. Mdh1
Official Symbol: Mdh1 and Name: Malate dehydrogenase 1 [Drosophila melanogaster (fruit fly)]
Other Aliases: Dmel_CG5362, CG5362, CT17038, Dmel\CG5362, MDH, MDH-1, Mdh, Mdh-1, Mdh2, MdhD, cMdh, sMdh
Other Designations: malate dehydrogenase 1; CG5362-PA; CG5362-PB; Mdh1-PA; Mdh1-PB; cytoplasmic Mdh; cytosolic malate dehydrogenase; malate dehydrogenase; malic DH
Chro

In [ ]:
for i in range(len(idlist2)):    
    handle2 = Entrez.efetch(db="nucleotide", id = idlist2[i], rettype = 'fasta', retmode = 'text', retnum=1)
    print(handle2.read())
    

>AK294428.1 Homo sapiens cDNA FLJ56506 complete cds, highly similar to Hexokinase-1 (EC 2.7.1.1)
GAGGAGCCGCCGAGCAGCCGCCGGAGGACCACGGCTCGCCAGGGCTGCGGAGGACCGACCGTCCCCACGC
CTGCCGCCCCGCGACCCCGACCGCCAGCATGATCGCCGCGCAGCTCCTGGCCTATTACTTCACGGAGCTG
AAGGATGACCAGGTCAAAAAGATTGACAAGTATCTCTATGCCATGCGGCTCTCCGATGAAACTCTCATAG
ATATCATGACTCGCTTCAGGAAGGAGATGAAGAATGGCCTCTCCCGGGATTTTAATCCAACAGCCACAGT
CAAGATGTTGCCAACATTCGTAAGGTCCATTCCTGATGGCTCTGAAAAGGGAGATTTCATTGCCCTGGAT
CTTGGTGGGTCTTCCTTTCGAATTCTGCGGGTGCAAGTGAATCATGAGAAAAACCAGAATGTTCACATGG
AGTCCGAGGTTTATGACACCCCAGAGAACATCGTGCACGGCAGTGGAAGCCAGCTTTTTGATCATGTTGC
TGAGTGCCTGGGAGATTTCATGGAGAAAAGGAAGATCAAGGACAAGAAGTTACCTGTGGGATTCACGTTT
TCTTTTCCTTGCCAACAATCCAAAATAGATGAGGCCATCCTGATCACCTGGACAAAGCGATTTAAAGCGA
GCGGAGTGGAAGGAGCAGATGTGGTCAAACTGCTTAACAAAGCCATCAAAAAGCGAGGGGACTATGATGC
CAACATCGTAGCTGTGGTGAATGACACAGTGGGCACCATGATGACCTGTGGCTATGACGACCAGCACTGT
GAAGTCGGCCTGATCATCGGCACTGGCACCAATGCTTGCTACATGGAGGAACTGAGGCACATTGATCTGG
TGGAAGGAGACGAGGGGAGGATGTGTATCAATACAGAATGGGGAGCCTTTG

>AK294396.1 Homo sapiens cDNA FLJ56429 complete cds, highly similar to Glucose-6-phosphate isomerase (EC 5.3.1.9)
ACACATATGTATCCACAGCGCCTAGAACACTGCCTGTAACGTGGAAGGTGTTCGATCTATAGAGTTTTGT
CGAATGAATGAATGAAGCCGACTAGTGCACAGGGAGTGCAGCGGCGCGATGGTAGCTCTCTGCAGCCTCC
AACACCTGGGCTCCAGTGATCCCCGGGCTCTGCCCACCCTCCCCACTGCCACTTCCGGGCAGAGGCCAGC
AAAGCGGCGGCGCAAGAGTCCCGCCATGGCCGCTCTCACCCGGGACCCCCAGTTCCAGAAGCTGCAGCAA
TGGTACCGCGAGCACCGCTCCGAGCTGAACCTGCGCCGCCTCTTCGATGCCAACAAGGACCGCTTCAACC
ACTTCAGCTTGACCCTCAACACCAACCATGGGCATATCCTGGTGGATTACTCCAAGAACCTGGTGACGGA
GGACGTGATGCGGATGCTGGTGGACTTGGCCAAGTCCAGGGGCGTGGAGGCCGCCCGGGAGCGGATGTTC
AATGGTGAGAAGATCAACTACACCGAGGGTCGAGCCGTGCTGCACGTGGCTCTGCGGAACCGGTCAAACA
CACCCATCCTGGTAGACGGCAAGGATGTGATGCCAGAGGTCAACAAGGTTCTGGACAAGATGAAGTCTTT
CTGCCAGGGACCCCTCATGGTGACTGAAGCCCTTAAGCCATACTCTTCAGGAGGTCCCCGCGTCTGGTAT
GTCTCCAACATTGATGGAACTCACATTGCCAAAACCCTGGCCCAGCTGAACCCCGAGTCCTCCCTGTTCA
TCATTGCCTCCAAGACCTTTACTACCCAGGAGACCATCACGAATGCAGAGACGGCGAAGGAGTGGTTTCT
CCAGGCGGCCAAGGATCCTTCTGCAGTGGCGAAG

>AK296457.1 Homo sapiens cDNA FLJ52314 complete cds, highly similar to Pyruvate dehydrogenase E1 component alpha subunit, somatic form, mitochondrial precursor (EC 1.2.4.1)
ACCCGCGTCGTGCCTCCTGGGTTGTGAGGAGTCGCCGCTGCCGCCACTGCCTGTGCTTCATGAGGAAGAT
GCTCGCCGCCGTCTCCCGCGTGCTGTCTGGCGCTTCTCAGAAGCCGGCAAGCAGAGTGCTGGTAGCATCC
CGTAATTTTGCAAATGATGCTACATTTGAAATTAAGAAATGTGACCTTCACCGGCTGGAAGAAGGCCCTC
CTGTCACAACAGTGCTCACCAGGGAGGATGGGCTCAAATACTACAGGATGATGCAGACTGTACGCCGAAT
GGAGTTGAAAGCAGATCAGCTGTATAAACAGAAAATTATTCGTGGTTTCTGTCACTTGTGTGATGGTCAG
GAAGCTTGCTGTGTGGGCCTGGAGGCCGGCATCAACCCCACAGACCATCTCATCACAGCCTACCGGGCTC
ACGGCTTTACTTTCACCCGGGGCCTTTCCGTCCGAGAAATTCTCGCAGAGCTTACAGGACGAAAAGGAGG
TTGTGCTAAAGGGAAAGGAGGATCGATGCACATGTATGCCAAGAACTTCTACGGGGGCAATGGCATCGTG
GGAGCGCAGGGCCAGATATTCGAAGCTTACAACATGGCAGCTTTGTGGAAATTACCTTGTATTTTCATCT
GTGAGAATAATCGCTATGGAATGGGAACGTCTGTTGAGAGAGCGGCAGCCAGCACTGATTACTACAAGAG
AGGCGATTTCATTCCTGGGCTGAGAGTGGATGGAATGGATATCCTGTGCGTCCGAGAGGCAACAAGGTTT
GCTGCTGCCTATTGTAGATCTGGGAAGGGGCCCATCCTGATGGAGC